# Doc2Query

In [9]:
import os

In [13]:
os.chdir("ir-lab-sose-2024-augsburg-information-retrieval")
print(os.getcwd())

/hpc/gpfs2/home/u/hofmange/ir-lab-sose-2024-augsburg-information-retrieval


In [19]:
from Doc2Query.Doc2Query import Doc2Query
import pyterrier as pt
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client

In [ ]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

In [21]:
# PyTerrier dataset to pass
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

AttributeError: module 'pyterrier' has no attribute 'get_dataset'

In [4]:
type(pt_dataset)

pyterrier.datasets.IRDSDataset

### Meine eigenen Functionen

In [23]:
# Create an instance of Doc2Query
Doc2Query = Doc2Query("mistralai/Mixtral-8x7B-v0.1")
#Doc2Query = Doc2Query("mistralai/Mixtral-8x7B-v0.1")


# Call the method with the required argument
#extended_pt_dataset = Doc2Query.expandDocumentsByQueries(pt_dataset)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like mistralai/Mixtral-8x7B-v0.1 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True)

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to(0)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### TODOs

In [ ]:
# Das im Docker anpassen, was sujin meinet
# Mal mein Model in VsCode ausprobieren

# PyTerrier Functionen ausprobieren

### Create dummy pyterrier dataset

In [2]:
import shutil
import os

index_dir = "./index"

# Remove the existing index directory
if os.path.exists(index_dir):
    shutil.rmtree(index_dir)

In [3]:
import pandas as pd
import pyterrier as pt

# Initialize PyTerrier
if not pt.started():
    pt.init()

# Sample data
documents = [
    {"docno": "1", "text": "The quick brown fox jumps over the lazy dog."},
    {"docno": "2", "text": "A journey of a thousand miles begins with a single step."},
    {"docno": "3", "text": "To be or not to be, that is the question."}
]

queries = [
    {"qid": "1", "query": "quick fox"},
    {"qid": "2", "query": "journey begins"}
]

relevance_judgments = [
    {"qid": "1", "docno": "1", "label": 1},
    {"qid": "2", "docno": "2", "label": 1}
]

# Create DataFrames
documents_df = pd.DataFrame(documents)
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(relevance_judgments)

# Prepare documents for indexing
# Convert DataFrame to list of dicts as required by IterDictIndexer
documents_list = documents_df.to_dict(orient='records')

# Index the documents using PyTerrier
indexer = pt.IterDictIndexer("./index")
indexref = indexer.index(documents_list, fields=["text"], meta=["docno"])

# Retrieve documents using BM25
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")

# Perform retrieval
results = bm25.transform(queries_df)

# Evaluate the results
eval = pt.Utils.evaluate(results, qrels_df, metrics=["map", "ndcg"])
print("Evaluation Metrics:")
print(eval)

/tmp/ipykernel_56782/877748803.py:36: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  indexref = indexer.index(documents_list, fields=["text"], meta=["docno"])


Evaluation Metrics:
{'map': 1.0, 'ndcg': 1.0}


/tmp/ipykernel_56782/877748803.py:45: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(results, qrels_df, metrics=["map", "ndcg"])


In [6]:
run = bm25(queries_df)
run = bm25.transform(queries_df)

print('Done. Here are the first 10 entries of the run')
run.head(10)

Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,1,0,1,0,1.273548,quick fox
1,2,1,2,0,1.273548,journey begins
